Description: 

this is the pseudo code of mapping the CONTCAR of DFT calculation to reference structure, which is consists 3 types of of Li: 48h type 1, 48h type 2, and 24g

# Initialization

## Import Libraries

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import os, sys
import shutil
from itertools import islice
from itertools import repeat
from addict import Dict
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import math
from collections import defaultdict

# pymatgen libraries
from pymatgen.core.structure import Structure
from pymatgen.transformations.standard_transformations import SupercellTransformation
from pymatgen.analysis.structure_matcher import StructureMatcher
from pymatgen.io.cif import CifWriter
from pymatgen.io.vasp.inputs import Poscar

from get_weirdos_ver3_new import splitall, copy_rename_single_file, copy_rename_files_savedir, check_folder_existance, get_structure_with_library, get_structure_with_linalg, get_structure_with_linalg_combinded_with_library, get_coor_structure24_dict_iterated, get_structure_with_linalg_orientated, get_orientated_positive_lessthan1_cif, get_orientated_positive_lessthan1_poscar, diagonalizing_latticeconstantsmatrix, get_coor_dict_structure, get_positive_lessthan1_poscarcontcar, mic_eucledian_distance_cartesian, get_flag_map_weirdos_el, rewrite_cif_w_correct_Li_idx, rewrite_cif_w_correct_Li_idx_weirdos_appended, format_spacing_cif, create_cif_pymatgen, ascending_Li, get_latticeconstant_structure_dict_iterated, get_fractional_to_cartesian_matrix, get_fractional_to_cartesian_coor
from get_weirdos_ver3_new import get_idx_coor_limapped_weirdos_dict, plot_energy_vs_latticeconstant
from get_weirdos_ver3_new import get_dx_dz_init
from get_weirdos_ver3_new import varying_radius_vs_sumweirdosLi, plot_varying_radius_vs_sumweirdosLi
from get_weirdos_ver3_new import get_flag_map_weirdos_48htype2_el, get_flag_map_weirdos_48htypesmerged_el, get_distance_weirdos_litype2_el, get_idx_weirdos_litype2_el, idx_correcting_mapped_litype2_el, create_combine_structure, get_distance_weirdos_litype2_label_el

direc = os.getcwd() # get current working directory

## Define Parameter

In [2]:
file_perfect_poscar_48n24 = "Li6PS5Cl_type2.cif"
max_mapping_radius = 0.043 
max_mapping_radius_48htype2 = 0.074
file_perfect_poscar_24 = "Li6PS5Cl_24_mod_2p27291.cif"

In [ ]:
path_perfect_poscar_24 = os.path.join(directory, file_perfect_poscar_24)
path_perfect_poscar_48n24 = os.path.join(directory, file_perfect_poscar_48n24)

ref_structure_48n24 = Structure.from_file(path_perfect_poscar_48n24)

file_loc_important_cols = pd.DataFrame() # dataframe contain all important information

# Orientation (if needed. not important now)

## CONTCAR of path 0 (perfect system) wrt structure_reference (24g)

In [8]:
def get_orientation():
        # get the structure of reference file 24g
        structure_reference = Structure.from_file(path_perfect_poscar_24)
        
        # get the scaling, translation, mapping of CONTCAR of path 0 (perfect system) wrt structure_reference (24g)
            # 3 different methods to compare. results are same
        get_structure_with_library()
        get_structure_with_linalg()
        get_structure_with_linalg_combinded_with_library()

        # then copy scaling, translation, mapping of path 0 to other paths of each corresponding geometry folder
        file_loc_important_cols["scaling"][idx] = file_loc_important_cols["scaling"][idx_path_0].copy()
        file_loc_important_cols["translation"][idx] = file_loc_important_cols["translation"][idx_path_0].copy()
        file_loc_important_cols["mapping"][idx] = file_loc_important_cols["mapping"][idx_path_0].copy()

    return file_loc_important_cols

## Other folders (NON perfect system)

In [ ]:
get_structure_with_linalg(file_loc_important_cols) # to all structure 

# Mapping

## Get coordinates of the structures as library: Li, P, S, Cl

In [ ]:
# reference structure
coor_structure_init_dict = get_coor_dict_structure(ref_structure_48n24)
# all structures geometry and path
get_coor_structure24_dict_iterated(file_loc_important_cols)

## turn coordinate x, y, z in CONTCAR into 0 < coor < 1

In [ ]:
def get_positive_lessthan1_poscarcontcar():
    df_positive_val = df[['coord_x', 'coord_y', 'coord_z']]
    for idx_a, coord_x in enumerate(df_positive_val['coord_x']):
        while float(coord_x) < 0.0:
            coord_x = float(coord_x) + 1.0
        while float(coord_x) > 1:
            coord_x = float(coord_x) - 1
        df_positive_val['coord_x'][idx_a] = '{:.{width}f}'.format(float(coord_x), width=n_decimal)

    for idx_a, coord_y in enumerate(df_positive_val['coord_y']):
        while float(coord_y) < 0.0:
            coord_y = float(coord_y) + 1.0
        while float(coord_y) > 1:
            coord_y = float(coord_y) - 1
        df_positive_val['coord_y'][idx_a] = '{:.{width}f}'.format(float(coord_y), width=n_decimal)

    for idx_a, coord_z in enumerate(df_positive_val['coord_z']):
        while float(coord_z) < 0.0:
            coord_z = float(coord_z) + 1.0
        while float(coord_z) > 1:
            coord_z = float(coord_z) - 1
        df_positive_val['coord_z'][idx_a] = '{:.{width}f}'.format(float(coord_z), width=n_decimal)



## Map with respect to all Li types (48h type 1, 48h type 2, and 24g) with the small max_mapping_radius (i.e. 0.043)

In [ ]:
def get_flag_map_weirdos_el(dataframe, coor_structure_init_dict, el, max_mapping_radius):
    # el = "Li". as we want to map Li only
    coor_origin120_el_init = coor_structure_init_dict[el]

    for idx in range(dataframe["geometry"].size):
        atom_mapping_el = {} 
        atom_mapping_el_w_dist = {} 
        coor_weirdos_el = []
        coor_weirdos_el_dict = {}

        # all CONTCAR with 0 < coor value < 1. take the Li only
        coor_origin24_el_init = dataframe.at[idx, "subdir_positive_CONTCAR"][el]
        
        coor_reduced120_el = coor_origin120_el_init.copy()
        coor_weirdos_el = coor_origin24_el_init.copy()    

        # outer loop: reference structure with 120 li-ions
        for idx120, coor120 in enumerate(coor_origin120_el_init):        
            counter = 0
            atom_mapping_w_dist_dict = {}
            distance_prev = float("inf")
            closest24 = None
            
            # inner loop: CONTCAR with 24 Li
            for idx24, coor24 in enumerate(coor_origin24_el_init):
                distance = mic_eucledian_distance(coor120, coor24)
                
                if distance < max_mapping_radius:
                    # get the coor with the closest distance
                    if distance < distance_prev:
                        distance_prev = distance
                        closest24 = coor24
                        counter = counter + 1

                # flag if any coor120 fits in more than one coor24 
                if counter > 1:
                    dataframe.at[idx, col_flag_el] = "True"
                
            if closest24 is not None:
                atom_mapping_w_dist_dict['closest24'] = tuple(closest24)
                atom_mapping_w_dist_dict['dist'] = distance_prev

                # create dictionary for coor120, its closest coor24, and distance
                if tuple(coor120) in atom_mapping_el_w_dist:
                    atom_mapping_el_w_dist[tuple(coor120)].append(atom_mapping_w_dist_dict)
                else:
                    atom_mapping_el_w_dist[tuple(coor120)] = atom_mapping_w_dist_dict
                    
                if tuple(coor120) in atom_mapping_el:
                    atom_mapping_el[tuple(coor120)].append(closest24)
                else:
                    atom_mapping_el[tuple(coor120)] = tuple(closest24)

                coor_weirdos_el = [arr for arr in coor_weirdos_el if not np.array_equal(arr, closest24)]

            # if no coor24 fits within the radius. delete the coor120. this is for the mapped CONTCAR
            if counter == 0:
                coor_reduced120_el = [arr for arr in coor_reduced120_el if not np.array_equal(arr, coor120)]

        # get any coor24 that has more than one coor120 fits within the radius
        duplicate_closest24_w_data = get_duplicate_closest24_w_data(atom_mapping_el_w_dist)

        # get the new reduced coor120, based on the closest distance if it has multiple close coor120 within the radius
        if len(duplicate_closest24_w_data) > 0:
            atom_mapping_el_w_dist_filtered = get_atom_mapping_el_w_dist_filtered(atom_mapping_el_w_dist)
            coor_reduced120_el_new = [list(key) for key in atom_mapping_el_w_dist_filtered.keys()]
        else:
            coor_reduced120_el_new = coor_reduced120_el

        sum_weirdos_el = len(coor_weirdos_el)
        sum_mapped_el = len(coor_reduced120_el)
        sum_mapped_el_new = len(coor_reduced120_el_new)

In [ ]:
# minimum image convection eucledian distance
def mic_eucledian_distance(coor1, coor2):
    x_coor1, y_coor1, z_coor1 = coor1
    x_coor2, y_coor2, z_coor2 = coor2
    
    delta_x = x_coor1 - x_coor2
    delta_y = y_coor1 - y_coor2
    delta_z = z_coor1 - z_coor2

    distance = math.sqrt(sum([(apply_pbc(delta_x))**2, (apply_pbc(delta_y))**2, (apply_pbc(delta_z))**2]))
    return distance

# periodic boundary condition
def apply_pbc(value):
    if abs(value) > 0.5:
        return 1 - abs(value)
    return value

In [ ]:
def get_duplicate_closest24_w_data(dict):
    duplicate_closest24 = {}
    for coor120, value in dict.items():
        closest24 = value["closest24"]
        dist = value["dist"]
        if closest24 in duplicate_closest24:
            duplicate_closest24[closest24].append({"coor120": coor120, "dist": dist})
        else:
            duplicate_closest24[closest24] = [{"coor120": coor120, "dist": dist}]

    duplicate_closest24_w_data = {}
    for closest24, coor120s_dists in duplicate_closest24.items():
        if len(coor120s_dists) > 1:
            duplicate_closest24_w_data[f"Duplicate closest24: {closest24}"] = [{"coor120s and dists": coor120s_dists}]

    return duplicate_closest24_w_data

In [ ]:
def get_atom_mapping_el_w_dist_filtered(dict):
    filtered_data = {}
    for coor120, value in dict.items():
        closest24 = value["closest24"]
        dist = value["dist"]
        if closest24 in filtered_data:
            if dist < filtered_data[closest24]["dist"]:
                filtered_data[closest24] = {"coor120": coor120, "dist": dist}
        else:
            filtered_data[closest24] = {"coor120": coor120, "dist": dist}

    atom_mapping_el_w_dist_filtered = {entry["coor120"]: {"closest24": key, "dist": entry["dist"]} for key, entry in filtered_data.items()}
    return atom_mapping_el_w_dist_filtered

## Mapping with larger radius and specifically only for Li 48h type 2
max_mapping_radius_48htype2 (i.e. 0.76)

In [ ]:
# basically same code like above. but different in input. 
# we use coor_weirdos_Li_dict as input instead of coordinate of CONTCAR structure
# for this I didn't rename the variable

def get_flag_map_weirdos_48htype2_el(dataframe, coor_structure_init_dict, el, max_mapping_radius_48htype2):
    coor_origin120_el_init = coor_structure_init_dict[el]                       

    coor_li48htype1_ref = coor_origin120_el_init[0:48]
    coor_li48htype2_ref = coor_origin120_el_init[48:96] 
    coor_li24g_ref = coor_origin120_el_init[96:120]

    for idx in range(dataframe["geometry"].size):
        atom_mapping_el = {} 
        atom_mapping_el_w_dist = {} 
        coor_weirdos_el = []

        coor_origin24_el_init = dataframe.at[idx, f"coor_weirdos_{el}_dict"][el]    # here is the difference !!!
        
        # ONLY process the one that has weirdos
        if len(coor_origin24_el_init) > 0:
            coor_reduced120_el = coor_li48htype2_ref.copy()
            coor_weirdos_el = coor_origin24_el_init.copy()    

            # outer loop: coordinate of Li 48h type 2
            for idx120, coor120 in enumerate(coor_li48htype2_ref):        
                counter = 0
                atom_mapping_w_dist_dict = {}
                distance_prev = float("inf")
                closest24 = None

                # inner loop: coordinate of weirdos
                for idx24, coor24 in enumerate(coor_origin24_el_init):
                    distance = mic_eucledian_distance(coor120, coor24)
                    
                    if distance < max_mapping_radius_48htype2:
                        if distance < distance_prev:
                            distance_prev = distance
                            closest24 = coor24
                            counter = counter + 1
                
                    if counter > 1:
                        dataframe.at[idx, col_flag_48htype2_el] = "True"

                # the rest are the same like before
                if closest24 is not None:
                    atom_mapping_w_dist_dict['closest24'] = tuple(closest24)
                    atom_mapping_w_dist_dict['dist'] = distance_prev

                    if tuple(coor120) in atom_mapping_el_w_dist:
                        atom_mapping_el_w_dist[tuple(coor120)].append(atom_mapping_w_dist_dict)
                    else:
                        atom_mapping_el_w_dist[tuple(coor120)] = atom_mapping_w_dist_dict
                        
                    if tuple(coor120) in atom_mapping_el:
                        atom_mapping_el[tuple(coor120)].append(closest24)
                    else:
                        atom_mapping_el[tuple(coor120)] = tuple(closest24)

                    coor_weirdos_el = [arr for arr in coor_weirdos_el if not np.array_equal(arr, closest24)]

                if counter == 0:
                    coor_reduced120_el = [arr for arr in coor_reduced120_el if not np.array_equal(arr, coor120)]

            duplicate_closest24_w_data = get_duplicate_closest24_w_data(atom_mapping_el_w_dist)

            if len(duplicate_closest24_w_data) > 0:
                atom_mapping_el_w_dist_filtered = get_atom_mapping_el_w_dist_filtered(atom_mapping_el_w_dist)
                coor_reduced120_el_new = [list(key) for key in atom_mapping_el_w_dist_filtered.keys()]
            else:
                coor_reduced120_el_new = coor_reduced120_el

            sum_weirdos_el = len(coor_weirdos_el)
            sum_mapped_el = len(coor_reduced120_el)
            sum_mapped_el_new = len(coor_reduced120_el_new)


## Merge the results of both mapping with different radius

In [ ]:
def get_flag_map_weirdos_48htypesmerged_el(dataframe, el):
    # columns from the 1st mapping to be called later
    col_atom_mapping_el = f"atom_mapping_{el}"
    col_atom_mapping_el_w_dist = f"atom_mapping_{el}_w_dist"
    col_duplicate_closest24_w_data_el = f"duplicate_closest24_w_data_{el}"
    col_coor_reduced120_el_new = f"coor_reduced120_{el}_new"
    col_sum_mapped_el_new = f"sum_mapped_{el}_new"
    col_sum_sanitycheck_el_new = f"sum_sanitycheck_{el}_new"

    # columns from the 2nd mapping to be called later
    col_atom_mapping_48htype2_el = f"atom_mapping_48htype2_{el}"
    col_atom_mapping_48htype2_el_w_dist = f"atom_mapping_48htype2_{el}_w_dist"
    col_coor_weirdos_48htype2_el = f"coor_weirdos_48htype2_{el}"
    col_sum_weirdos_48htype2_el = f"sum_weirdos_48htype2_{el}"
    col_duplicate_closest24_w_data_48htype2_el = f"duplicate_closest24_w_data_48htype2_{el}"
    col_coor_reduced120_48htype2_el_new = f"coor_reduced120_48htype2_{el}_new"
    col_sum_mapped_48htype2_el_new = f"sum_mapped_48htype2_{el}_new"
    col_sum_sanitycheck_48htype2_el_new = f"sum_sanitycheck_48htype2_{el}_new"

    for idx in range(dataframe["geometry"].size):
        # 1st atom mapping results. to be merged.
        atom_mapping_el = dataframe.at[idx, col_atom_mapping_el]
        atom_mapping_el_w_dist = dataframe.at[idx, col_atom_mapping_el_w_dist]
        coor_reduced120_el_new = dataframe.at[idx, col_coor_reduced120_el_new]
        # 2nd atom mapping results. to be merged.
        atom_mapping_48htype2_el = dataframe.at[idx, col_atom_mapping_48htype2_el]
        atom_mapping_48htype2_el_w_dist = dataframe.at[idx, col_atom_mapping_48htype2_el_w_dist]
        coor_reduced120_48htype2_el_new = dataframe.at[idx, col_coor_reduced120_48htype2_el_new]

        # these are the absolute result. no need further merging.
        coor_weirdos_48htypesmerged_el = dataframe.at[idx, col_coor_weirdos_48htype2_el]
        sum_weirdos_48htypesmerged_el = dataframe.at[idx, col_sum_weirdos_48htype2_el]
        duplicate_closest24_w_data_48htypesmerged = dataframe.at[idx, col_duplicate_closest24_w_data_48htype2_el]
        
        # initializing list for merging
        coor_reduced120_48htypesmerged_el_new = []

        # merge the mapping dictionary from both mapping results
        # mapping dictionary has coor120 as key, coor24 as value
        atom_mapping_48htypesmerged_el = merge_dictionaries(atom_mapping_el, atom_mapping_48htype2_el)
        # check if after the merging there are double coor24
        duplicate_coor24s = check_duplicatevalues(atom_mapping_48htypesmerged_el)
        if len(duplicate_coor24s) > 1:        
            dataframe.at[idx, f"flag_48htypesmerged_{el}"] = "True"
        
        # merging mapping results with distance
        atom_mapping_48htypesmerged_el_w_dist = merge_dictionaries(atom_mapping_el_w_dist, atom_mapping_48htype2_el_w_dist)

        # merge the mapped coordinate from both mapping results
        if coor_reduced120_48htype2_el_new.ndim == coor_reduced120_el_new.ndim:
            coor_reduced120_48htypesmerged_el_new = np.concatenate((coor_reduced120_el_new, coor_reduced120_48htype2_el_new), axis=0)
        # if 2nd mapping results in no coordinate, then copy the result of the 1st mapping
        elif coor_reduced120_48htype2_el_new.ndim == 1:
            coor_reduced120_48htypesmerged_el_new = np.array(coor_reduced120_el_new.copy())
        # accordingly for 1st mapping results
        elif coor_reduced120_el_new.ndim == 1:
            coor_reduced120_48htypesmerged_el_new = np.array(coor_reduced120_48htype2_el_new.copy())
        else:
            print(f"coor_reduced120_48htype2_el_new or coor_reduced120_el_new has no correct dimension at idx: {idx}")

        sum_mapped_48htypesmerged_el_new = len(coor_reduced120_48htypesmerged_el_new)


In [ ]:
# merge two dictionaries
def merge_dictionaries(dict1, dict2):
    merged_dict = defaultdict(list)

    for d in (dict1, dict2): 
        for key, value in d.items():
            merged_dict[key].append(value)
    
    return merged_dict

# check duplicate values
def check_duplicate_values(dictionary):
    seen_values = set()
    duplicate_values = set()

    for value in dictionary.values():
        value_tuple = tuple(value) 
        if value_tuple in seen_values:
            duplicate_values.add(value_tuple)
        else:
            seen_values.add(value_tuple)

    return duplicate_values

## get distance of weirdos, its top 3 closest coor120, and label

In [ ]:
def get_distance_weirdos_litype2_label_el(dataframe, coor_structure_init_dict, el, litype2):
    # to do: add idx of weirdo and coor120
    coor_origin120_el_init = coor_structure_init_dict[el]

    coor_li48htype1_ref = coor_origin120_el_init[0:48]
    coor_li48htype2_ref = coor_origin120_el_init[48:96]
    coor_li24g_ref = coor_origin120_el_init[96:120]

    for idx in range(dataframe["geometry"].size):
        dist_weirdos_atom120_el = []
        dist_weirdos_el = []
        coorweirdo_dist_label_coor120_idxweirdo_idx120_el = {}

        # coordinate of weirdos
        coor_weirdos_el = dataframe.at[idx, f"coor_weirdos_48htype2_{el}"]

        # process only if weirdo exists
        if len(coor_weirdos_el) > 0:

            # outer loop: coor weirdo
            for coor_weirdo in coor_weirdos_el:
                distance_weirdo_prev = float('inf')

                coorweirdo_dist_label_coor120_idxweirdo_idx120_el[tuple(coor_weirdo)] = []
                
                # inner loop: reference structure with 120 li-ions
                for idx120, coor120 in enumerate(coor_origin120_el_init):
                    coorweirdo_dist_label_coor120_val_el = {}
            
                    distance_weirdo = mic_eucledian_distance(coor120, coor_weirdo)

                    coorweirdo_dist_label_coor120_val_el['dist'] = distance_weirdo

                    # get label of 48htype1 and create dictionary = key: coor_weirdo; values: distance, type, coor120
                    for idx_li48htype1_temp, coor_li48htype1_ref_temp in enumerate(coor_li48htype1_ref):
                        if (coor120 == coor_li48htype1_ref_temp).all():
                            coorweirdo_dist_label_coor120_val_el["type"] = "48htype1"
                            coorweirdo_dist_label_coor120_val_el["coor120"] = tuple(coor120)
                            if tuple(coor_weirdo) in coorweirdo_dist_label_coor120_idxweirdo_idx120_el:
                                coorweirdo_dist_label_coor120_idxweirdo_idx120_el[tuple(coor_weirdo)].append(coorweirdo_dist_label_coor120_val_el)
                            else:
                                coorweirdo_dist_label_coor120_idxweirdo_idx120_el[tuple(coor_weirdo)] = coorweirdo_dist_label_coor120_val_el
                    # get label of 48htype2
                    for idx_li48htype2_temp, coor_li48htype2_ref_temp in enumerate(coor_li48htype2_ref):
                        if (coor120 == coor_li48htype2_ref_temp).all():
                            coorweirdo_dist_label_coor120_val_el["type"] = "48htype2"
                            coorweirdo_dist_label_coor120_val_el["coor120"] = tuple(coor120)
                            if tuple(coor_weirdo) in coorweirdo_dist_label_coor120_idxweirdo_idx120_el:
                                coorweirdo_dist_label_coor120_idxweirdo_idx120_el[tuple(coor_weirdo)].append(coorweirdo_dist_label_coor120_val_el)
                            else:
                                coorweirdo_dist_label_coor120_idxweirdo_idx120_el[tuple(coor_weirdo)] = coorweirdo_dist_label_coor120_val_el
                    # get label of 24g
                    for idx_24g_temp, coor_li24g_ref_temp in enumerate(coor_li24g_ref):
                        if (coor120 == coor_li24g_ref_temp).all():
                            coorweirdo_dist_label_coor120_val_el["type"] = "24g"
                            coorweirdo_dist_label_coor120_val_el["coor120"] = tuple(coor120)
                            if tuple(coor_weirdo) in coorweirdo_dist_label_coor120_idxweirdo_idx120_el:
                                coorweirdo_dist_label_coor120_idxweirdo_idx120_el[tuple(coor_weirdo)].append(coorweirdo_dist_label_coor120_val_el)
                            else:
                                coorweirdo_dist_label_coor120_idxweirdo_idx120_el[tuple(coor_weirdo)] = coorweirdo_dist_label_coor120_val_el

                # sort dictionary based on asceding distance value
                sorted_coorweirdo_dist_label_coor120_idxweirdo_idx120_el = {
                                                                    k: sorted(v, key=lambda x: x['dist'])
                                                                    for k, v in coorweirdo_dist_label_coor120_idxweirdo_idx120_el.items()
                                                                }
                
                # take top3 distance and its values
                top3_sorted_coorweirdo_dist_label_coor120_idxweirdo_idx120_el = {k: v[0:3] for k, v in sorted_coorweirdo_dist_label_coor120_idxweirdo_idx120_el.items()}


## get index of weirdos

In [ ]:
def get_idx_weirdos_litype2_el(dataframe, el, litype2):

    for idx in range(dataframe["geometry"].size):
        coor_origin24_el_init = dataframe.at[idx, "subdir_positive_CONTCAR"][el]
        
        idx_weirdos_el = []

        # outer loop: CONTCAR 
        for i, given_arr in enumerate(coor_origin24_el_init):
            
            coor_weirdos_el = dataframe.at[idx, f"coor_weirdos_48htype2_{el}"]

            # process only if weirdo exists
            if len(coor_weirdos_el) > 0:

                # inner loop: coor of weirdo 
                for arr in coor_weirdos_el:
                    # get the index based on CONTCAR
                    if (arr == given_arr).all():
                        idx_weirdos_el.append(i)

        idx0_weirdos_el = idx_weirdos_el
        sum_weirdos_el = len(idx_weirdos_el)

## Correcting the index of the mapped li-ion

In [ ]:
def idx_correcting_mapped_litype2_el(dataframe, el, litype2):

    for idx in range(dataframe["geometry"].size):
        coor_origin24_el_init = dataframe.at[idx, "subdir_positive_CONTCAR"][el] 
        coor_reduced120_el_new = dataframe.at[idx, f"coor_reduced120_48htypesmerged_{el}_new"]

        idx_correcting_el = []
        atom_mapping_el_w_dist_idx24 = {} 
        idx_coor_el = {}

        # outer loop: merged coor of 1st and 2nd mapping
        for idx120, coor120 in enumerate(coor_reduced120_el_new):
            distance_prev = float("inf")
            closest24 = None
            idx_closest24 = None
            atom_mapping_w_dist_idx24_dict = {}

            atom_mapping_el_w_dist_idx24[tuple(coor120)] = []

            # inner loop: CONTCAR
            for idx24, coor24 in enumerate(coor_origin24_el_init):
                distance = mic_eucledian_distance(coor120, coor24)

                if distance != 0:
                    if distance < distance_prev:
                        distance_prev = distance
                        closest24 = coor24
                        idx_closest24 = idx24
            
            idx_correcting_el.append(idx_closest24)

            # create dictionary: coor120, closest24, dist, idx_closest24
            if closest24 is not None:
                atom_mapping_w_dist_idx24_dict['closest24'] = tuple(closest24)
                atom_mapping_w_dist_idx24_dict['dist'] = distance
                atom_mapping_w_dist_idx24_dict['idx_closest24'] = idx_closest24

                if tuple(coor120) in atom_mapping_el_w_dist_idx24:
                    atom_mapping_el_w_dist_idx24[tuple(coor120)].append(atom_mapping_w_dist_idx24_dict)
                else:
                    atom_mapping_el_w_dist_idx24[tuple(coor120)] = atom_mapping_w_dist_idx24_dict
        
        # correcting the index
        for i in range(len(idx_correcting_el)):
            idx_coor_el[idx_correcting_el[i]] = coor_reduced120_el_new[i]
        sorted_idx_coor_el = {key: val for key, val in sorted(idx_coor_el.items())}
        sorted_coor = list(sorted_idx_coor_el.values())


# Writing as CIF file (not yet summarized as pseudocode)

## Create combine structure

In [ ]:
def create_combine_structure(dataframe, destination_directory, amount_Li, amount_P, amount_S, var_savefilename):

    for idx in range(dataframe["geometry"].size):
        coor_combined = []

        new_structure = Structure.from_file(dataframe['subdir_CONTCAR'][idx])
        coor_origin24_init = dataframe.at[idx, "subdir_positive_CONTCAR"]
        coor_reduced120_Li = dataframe.at[idx, "coor_reduced120_corrected_Li_new"]

        coor_structure_init_P = coor_origin24_init["P"]
        coor_structure_init_S = coor_origin24_init["S"]
        coor_structure_init_Cl = coor_origin24_init["Cl"]

        coor_mapped_Li = np.array(coor_reduced120_Li)
        coor_origin_P = np.array(coor_structure_init_P)
        coor_origin_S = np.array(coor_structure_init_S)
        coor_origin_Cl = np.array(coor_structure_init_Cl)
    
        ## get the combined coordinate of the mapped Li with all other original elements
        for m in coor_mapped_Li:
            coor_combined.append(np.array(m))
        for n in coor_origin_P:
            coor_combined.append(np.array(n))
        for o in coor_origin_S:
            coor_combined.append(np.array(o))
        for p in coor_origin_Cl:
            coor_combined.append(np.array(p))
        
        coor_combined_array = [arr.tolist() for arr in coor_combined]

        ## getting the index
        amount_Li_temp = len(coor_reduced120_Li)
        amount_P_temp = len(coor_structure_init_P)
        amount_S_temp = len(coor_structure_init_S)
        amount_Cl_temp = len(coor_structure_init_Cl)

        # TO DO: write manually to change the line with Li{i}, where i = #of index based on the orientated positive
        idx_mapped_Li = np.arange(amount_Li_temp)
        idx_origin_P = np.arange(amount_P_temp) + amount_Li
        idx_origin_S = np.arange(amount_S_temp) + amount_Li + amount_P
        idx_origin_Cl = np.arange(amount_Cl_temp) + amount_Li + amount_P + amount_S

        idx_species_combined_idx0 = np.concatenate((idx_mapped_Li, idx_origin_P, idx_origin_S, idx_origin_Cl))
        idx_species_combined_idx0_int = idx_species_combined_idx0.astype(int).tolist()

        ## creating the structure file of combined elements
        selected_species_combined = [new_structure.species[i] for i in idx_species_combined_idx0_int]
        structure_combined = Structure(new_structure.lattice, selected_species_combined, coor_combined_array)
        cif_combined = CifWriter(structure_combined)
        source_filename = f"{int(dataframe['geometry'][idx])}_{int(dataframe['path'][idx])}_{var_savefilename}.cif"
        source_filename_path = os.path.join(destination_directory, source_filename)
        cif_combined.write_file(source_filename_path)


## rewrite CIF file with correct Li index

In [ ]:
def rewrite_cif_w_correct_Li_idx(dataframe, destination_directory, amount_Li, amount_P, amount_S, amount_Cl, var_savefilename_init, var_savefilename_new):
    col_idx_without_weirdos = "idx_without_weirdos"

    dataframe[col_idx_without_weirdos] = [np.array([]) for _ in range(len(dataframe.index))]
    
    search_string = "Li  Li0"

    for idx in range(dataframe["geometry"].size):
        idx0_weirdos_Li = dataframe["idx0_weirdos_Li"][idx]
        source_filename = f"{int(dataframe['geometry'][idx])}_{int(dataframe['path'][idx])}_{var_savefilename_init}.cif"
        source_filename_path = os.path.join(destination_directory, source_filename)

        source_filename_filtered = f"{int(dataframe['geometry'][idx])}_{int(dataframe['path'][idx])}_{var_savefilename_new}.cif"
        destination_path_combined_new = os.path.join(destination_directory, source_filename_filtered)

        with open(source_filename_path, "r") as f:
            lines = f.readlines()

        for idx_line, line in enumerate(lines):
            if search_string in line:
                idx_Li_start = idx_line
                break

        idx_without_weirdos = [i for i in range(amount_Li) if i not in idx0_weirdos_Li]

        new_text = []
        for i in range(len(idx_without_weirdos)):
            idx_line = idx_Li_start + i
            if lines[idx_line].strip().startswith("Li"):
                new_label = f"Li{idx_without_weirdos[i]}"
                modified_line = lines[idx_line].replace(lines[idx_line].split()[1], new_label)
                new_text.append(modified_line)

        lines[idx_Li_start : len(idx_without_weirdos) + idx_Li_start] = new_text

        idx_P_S_Cl_line_new_start    = len(idx_without_weirdos) + idx_Li_start
        reindex_P_S_Cl(lines, idx_Li_start, idx_without_weirdos, idx_P_S_Cl_line_new_start, amount_Li, amount_P, amount_S, amount_Cl)

        dataframe.at[idx, col_idx_without_weirdos] = idx_without_weirdos

        with open(destination_path_combined_new, "w") as f:
            f.write("\n".join(line.strip() for line in lines))


In [ ]:
def reindex_P_S_Cl(lines, idx_Li_start, idx_without_weirdos, idx_P_S_Cl_line_new_start, amount_Li, amount_P, amount_S, amount_Cl):
    old_text_P_S_Cl = lines[len(idx_without_weirdos) + idx_Li_start :]

    idx_P_S_Cl_line_new_end      = idx_P_S_Cl_line_new_start + len(old_text_P_S_Cl)
    lines[idx_P_S_Cl_line_new_start : idx_P_S_Cl_line_new_end] = old_text_P_S_Cl

    # re-write the index of P_S_Cl accordingly
    new_text_P_S_Cl = []
    for i in range(amount_P):
        idx_line_P = idx_P_S_Cl_line_new_start + i
        idx_P_new = amount_Li + i
        if lines[idx_line_P].strip().startswith("P"):
            new_label = f"P{idx_P_new}"
            modified_line = lines[idx_line_P].replace(lines[idx_line_P].split()[1], new_label)
            new_text_P_S_Cl.append(modified_line)
    for i in range(amount_S):
        idx_line_S = idx_P_S_Cl_line_new_start + amount_P + i
        idx_S_new = amount_Li + amount_P + i
        if lines[idx_line_S].strip().startswith("S"):
            new_label = f"S{idx_S_new}"
            modified_line = lines[idx_line_S].replace(lines[idx_line_S].split()[1], new_label)
            new_text_P_S_Cl.append(modified_line)
    for i in range(amount_Cl):
        idx_line_Cl = idx_P_S_Cl_line_new_start + amount_P + amount_S + i
        idx_Cl_new = amount_Li + amount_P + amount_S + i
        if lines[idx_line_Cl].strip().startswith("Cl"):
            new_label = f"Cl{idx_Cl_new}"
            modified_line = lines[idx_line_Cl].replace(lines[idx_line_Cl].split()[1], new_label)
            new_text_P_S_Cl.append(modified_line)

    lines[idx_P_S_Cl_line_new_start : amount_P + amount_S + amount_Cl + idx_P_S_Cl_line_new_start] = new_text_P_S_Cl

    return lines

## Appending weirdos in CIF (for visualization)

In [ ]:
def rewrite_cif_w_correct_Li_idx_weirdos_appended(dataframe, destination_directory, amount_Li, amount_P, amount_S, amount_Cl, var_savefilename_init, var_savefilename_new):
    
    search_string = "Li  Li0"

    for idx in range(dataframe["geometry"].size):
        idx0_weirdos_Li = dataframe["idx0_weirdos_Li"][idx]
        source_filename = f"{int(dataframe['geometry'][idx])}_{int(dataframe['path'][idx])}_{var_savefilename_init}.cif"
        source_filename_path = os.path.join(destination_directory, source_filename)

        source_filename_filtered = f"{int(dataframe['geometry'][idx])}_{int(dataframe['path'][idx])}_{var_savefilename_new}.cif"
        destination_path_combined_new = os.path.join(destination_directory, source_filename_filtered)

        with open(source_filename_path, "r") as f:
            lines = f.readlines()

        for idx_line, line in enumerate(lines):
            if search_string in line:
                idx_Li_start = idx_line
                break

        idx_without_weirdos = [i for i in range(amount_Li) if i not in idx0_weirdos_Li]

        new_text = []
        for i in range(len(idx_without_weirdos)):
            idx_line = idx_Li_start + i
            if lines[idx_line].strip().startswith("Li"):
                new_label = f"Li{idx_without_weirdos[i]}"
                modified_line = lines[idx_line].replace(lines[idx_line].split()[1], new_label)
                new_text.append(modified_line)

        lines[idx_Li_start : len(idx_without_weirdos) + idx_Li_start] = new_text

        # now appending weirdos below existing lines of Li
        coor_weirdos = dataframe["coor_weirdos_48htypesmerged_Li"][idx] # coor_weirdos = dataframe["coor_weirdos_Li"][idx]

        weirdos_text = []
        for i in range(len(idx0_weirdos_Li)):
            coor_weirdo_x = coor_weirdos[i][0]
            coor_weirdo_y = coor_weirdos[i][1]
            coor_weirdo_z = coor_weirdos[i][2]
            idx_weirdo = idx0_weirdos_Li[i]
            new_line_weirdo = f"Li  Li{idx_weirdo}  1  {coor_weirdo_x:.8f}  {coor_weirdo_y:.8f}  {coor_weirdo_z:.8f}  1"  # manually created
            weirdos_text.append(new_line_weirdo)
            # idx_line_weirdos = idx_Li_start + len(idx_without_weirdos)
            
        old_text_P_S_Cl = lines[len(idx_without_weirdos) + idx_Li_start :]

        idx_weirdo_line_start   = len(idx_without_weirdos) + idx_Li_start
        idx_weirdo_line_end     = idx_weirdo_line_start + len(idx0_weirdos_Li)
        lines[idx_weirdo_line_start : idx_weirdo_line_end] = weirdos_text

        idx_P_S_Cl_line_new_start    = idx_weirdo_line_end

        # !!!: for the moment not using the function because P is gone
        # reindex_P_S_Cl(lines, idx_Li_start, idx_without_weirdos, idx_P_S_Cl_line_new_start, amount_P, amount_S, amount_Cl)
        
        idx_P_S_Cl_line_new_end      = idx_P_S_Cl_line_new_start + len(old_text_P_S_Cl)
        lines[idx_P_S_Cl_line_new_start : idx_P_S_Cl_line_new_end] = old_text_P_S_Cl
 
        # re-write the index of P_S_Cl accordingly
        new_text_P_S_Cl = []
        for i in range(amount_P):
            idx_line_P = idx_P_S_Cl_line_new_start + i
            idx_P_new = amount_Li + i
            if lines[idx_line_P].strip().startswith("P"):
                new_label = f"P{idx_P_new}"
                modified_line = lines[idx_line_P].replace(lines[idx_line_P].split()[1], new_label)
                new_text_P_S_Cl.append(modified_line)
        for i in range(amount_S):
            idx_line_S = idx_P_S_Cl_line_new_start + amount_P + i
            idx_S_new = amount_Li + amount_P + i
            if lines[idx_line_S].strip().startswith("S"):
                new_label = f"S{idx_S_new}"
                modified_line = lines[idx_line_S].replace(lines[idx_line_S].split()[1], new_label)
                new_text_P_S_Cl.append(modified_line)
        for i in range(amount_Cl):
            idx_line_Cl = idx_P_S_Cl_line_new_start + amount_P + amount_S + i
            idx_Cl_new = amount_Li + amount_P + amount_S + i
            if lines[idx_line_Cl].strip().startswith("Cl"):
                new_label = f"Cl{idx_Cl_new}"
                modified_line = lines[idx_line_Cl].replace(lines[idx_line_Cl].split()[1], new_label)
                new_text_P_S_Cl.append(modified_line)

        lines[idx_P_S_Cl_line_new_start : amount_P + amount_S + amount_Cl + idx_P_S_Cl_line_new_start] = new_text_P_S_Cl

        # dataframe.at[idx, col_subdir_cif_w_correct_Li_idx_weirdos_appended] = destination_path_combined_new

        # Write the modified lines back to the file
        with open(destination_path_combined_new, "w") as f:
            f.write("\n".join(line.strip() for line in lines))


## Summary of all atoms in CONTCAR: type (48h type 1, 48h type 2, weirdo); amount of each type and weirdo

In [ ]:
def get_idx_coor_limapped_weirdos_dict(dataframe, coor_structure_init_dict, el):
    coor_origin120_el_init = coor_structure_init_dict[el]

    col_idx_without_weirdos = "idx_without_weirdos"
    col_coor_reduced120_corrected_Li_new = "coor_reduced120_corrected_Li_new"
    col_coor_weirdos_48htypesmerged_Li = "coor_weirdos_48htypesmerged_Li"
    col_idx0_weirdos_Li = "idx0_weirdos_Li"
    col_nr_of_weirdos_Li = f"#weirdos_Li"
    col_sum_sanitycheck_48htypesmerged_Li_new = "sum_sanitycheck_48htypesmerged_Li_new"

    col_idx_coor_limapped_weirdos_dict = "idx_coor_limapped_weirdos_dict"
    col_sum_label_and_weirdo_flag = "#label_and_#weirdo_flag"
    col_amount_types_and_weirdo = "amount_types_and_weirdo"
    col_ratio_48htype1_Li = "ratio_48htype1_Li"
    col_ratio_48htype2_Li = "ratio_48htype2_Li"
    col_ratio_24g_Li = "ratio_24g_Li"
    col_ratio_weirdo_Li = "ratio_weirdo_Li"
    col_sum_amount = "sum_amount"
    col_idx_coor_limapped_weirdos_dict_init = "idx_coor_limapped_weirdos_dict_init"
    col_ndim_coor_reduced120_corrected_Li_new = "ndim_coor_reduced120_corrected_Li_new"
    col_ndim_coor_weirdos_48htypesmerged_Li = "ndim_coor_weirdos_48htypesmerged_Li"
    col_len_coor_weirdos_48htypesmerged_Li = "len_coor_weirdos_48htypesmerged_Li"
    col_len_coor_reduced120_corrected_Li_new = "len_coor_reduced120_corrected_Li_new"
    col_len_idx0_weirdos_Li = "len_idx0_weirdos_Li"
    col_len_idx_without_weirdos = "len_idx_without_weirdos"
    col_ndim_flag_coor = "ndim_flag_coor"

    dataframe[col_idx_coor_limapped_weirdos_dict] = [{} for _ in range(len(dataframe.index))]
    dataframe[col_sum_label_and_weirdo_flag] = "False"
    dataframe[col_amount_types_and_weirdo] = [0 for _ in range(len(dataframe.index))]
    dataframe[col_sum_amount] = [0 for _ in range(len(dataframe.index))]
    dataframe[col_idx_coor_limapped_weirdos_dict_init] = [{} for _ in range(len(dataframe.index))]
    dataframe[col_ndim_coor_reduced120_corrected_Li_new] = [0 for _ in range(len(dataframe.index))]
    dataframe[col_ndim_coor_weirdos_48htypesmerged_Li] = [0 for _ in range(len(dataframe.index))]
    dataframe[col_len_coor_weirdos_48htypesmerged_Li] = [0 for _ in range(len(dataframe.index))]
    dataframe[col_len_coor_reduced120_corrected_Li_new] = [0 for _ in range(len(dataframe.index))]
    dataframe[col_len_idx0_weirdos_Li] = [0 for _ in range(len(dataframe.index))]
    dataframe[col_len_idx_without_weirdos] = [0 for _ in range(len(dataframe.index))]
    dataframe[col_ndim_flag_coor] = "False"

    coor_li48htype1_ref = coor_origin120_el_init[0:48]
    coor_li48htype2_ref = coor_origin120_el_init[48:96]
    coor_li24g_ref = coor_origin120_el_init[96:120]

    for idx in range(dataframe["geometry"].size):
        coor_limapped_weirdos = []
        idx0_limapped_weirdos = []
        idx_coor_limapped_weirdos_dict_init = {}
        idx_coor_limapped_weirdos_dict = {}

        idx_without_weirdos = dataframe.at[idx, col_idx_without_weirdos]
        coor_reduced120_corrected_Li_new = np.array(dataframe.at[idx, col_coor_reduced120_corrected_Li_new])
        coor_weirdos_48htypesmerged_Li = np.array(dataframe.at[idx, col_coor_weirdos_48htypesmerged_Li])
        idx0_weirdos_Li = dataframe.at[idx, col_idx0_weirdos_Li]
        nr_of_weirdos_Li = dataframe.at[idx, col_nr_of_weirdos_Li]
        sum_sanitycheck_48htypesmerged_Li_new = dataframe.at[idx, col_sum_sanitycheck_48htypesmerged_Li_new]

        ndim_coor_reduced120_corrected_Li_new = coor_reduced120_corrected_Li_new.ndim
        ndim_coor_weirdos_48htypesmerged_Li = coor_weirdos_48htypesmerged_Li.ndim
        len_coor_weirdos_48htypesmerged_Li = len(coor_weirdos_48htypesmerged_Li)
        len_coor_reduced120_corrected_Li_new = len(coor_reduced120_corrected_Li_new)
        len_idx0_weirdos_Li = len(idx0_weirdos_Li)
        len_idx_without_weirdos = len(idx_without_weirdos)
    
        if ndim_coor_reduced120_corrected_Li_new == ndim_coor_weirdos_48htypesmerged_Li & ndim_coor_weirdos_48htypesmerged_Li == 2:
            coor_limapped_weirdos = np.concatenate((coor_reduced120_corrected_Li_new, coor_weirdos_48htypesmerged_Li), axis=0)
            dataframe.at[idx, col_ndim_flag_coor] = "True"
        elif ndim_coor_weirdos_48htypesmerged_Li == 1:
            coor_limapped_weirdos = np.array(coor_reduced120_corrected_Li_new.copy())
        elif ndim_coor_reduced120_corrected_Li_new == 1:
            coor_limapped_weirdos = np.array(coor_weirdos_48htypesmerged_Li.copy())
        else:
            print(f"coor_weirdos_48htypesmerged_Li or coor_reduced120_corrected_Li_new has no correct dimension at idx: {idx}")

        if len(idx0_weirdos_Li) > 0:
            idx0_limapped_weirdos = np.concatenate((idx_without_weirdos, idx0_weirdos_Li), axis=0)
        elif len(idx0_weirdos_Li) == 0:
            idx0_limapped_weirdos = np.array(idx_without_weirdos.copy())
        elif len(idx_without_weirdos) == 0:
            idx0_limapped_weirdos = np.array(idx0_weirdos_Li.copy())
        else:
            print(f"idx0_weirdos_Li or idx_without_weirdos has no correct len at idx: {idx}")

        idx_coor_limapped_weirdos_dict_init = dict(zip(idx0_limapped_weirdos, coor_limapped_weirdos))

        coor_48htype1_Li = []; coor_48htype2_Li = []; coor_24g_Li = []; coor_weirdo_Li = []
        for key, value in idx_coor_limapped_weirdos_dict_init.items():
            idx_coor_limapped_weirdos_dict_val = {}

            idx_coor_limapped_weirdos_dict_val['coor'] = tuple(value)

            for idx_li48htype1_temp, coor_li48htype1_ref_temp in enumerate(coor_li48htype1_ref):
                if (value == coor_li48htype1_ref_temp).all():
                    idx_coor_limapped_weirdos_dict_val["type"] = "48htype1"
                    coor_48htype1_Li.append(np.array(list(value)))
                    if int(key) in idx_coor_limapped_weirdos_dict:
                        idx_coor_limapped_weirdos_dict[int(key)].append(idx_coor_limapped_weirdos_dict_val)
                    else:
                        idx_coor_limapped_weirdos_dict[int(key)] = idx_coor_limapped_weirdos_dict_val
            for idx_li48htype2_temp, coor_li48htype2_ref_temp in enumerate(coor_li48htype2_ref):
                if (value == coor_li48htype2_ref_temp).all():
                    idx_coor_limapped_weirdos_dict_val["type"] = "48htype2"
                    coor_48htype2_Li.append(np.array(list(value)))
                    if int(key) in idx_coor_limapped_weirdos_dict:
                        idx_coor_limapped_weirdos_dict[int(key)].append(idx_coor_limapped_weirdos_dict_val)
                    else:
                        idx_coor_limapped_weirdos_dict[int(key)] = idx_coor_limapped_weirdos_dict_val
            for idx_24g_temp, coor_li24g_ref_temp in enumerate(coor_li24g_ref):
                if (value == coor_li24g_ref_temp).all():
                    idx_coor_limapped_weirdos_dict_val["type"] = "24g"
                    coor_24g_Li.append(np.array(list(value)))
                    if int(key) in idx_coor_limapped_weirdos_dict:
                        idx_coor_limapped_weirdos_dict[int(key)].append(idx_coor_limapped_weirdos_dict_val)
                    else:
                        idx_coor_limapped_weirdos_dict[int(key)] = idx_coor_limapped_weirdos_dict_val
            for idx_weirdos_temp, coor_weirdos_ref_temp in enumerate(coor_weirdos_48htypesmerged_Li):
                if (value == coor_weirdos_ref_temp).all():
                    idx_coor_limapped_weirdos_dict_val["type"] = "weirdos"
                    coor_weirdo_Li.append(np.array(list(value)))
                    if int(key) in idx_coor_limapped_weirdos_dict:
                        idx_coor_limapped_weirdos_dict[int(key)].append(idx_coor_limapped_weirdos_dict_val)
                    else:
                        idx_coor_limapped_weirdos_dict[int(key)] = idx_coor_limapped_weirdos_dict_val


        # amount of each type
        amount_48htype1_Li = len(coor_48htype1_Li)
        amount_48htype2_Li = len(coor_48htype2_Li)
        amount_24g_Li = len(coor_24g_Li)
        amount_weirdo = len(coor_weirdo_Li)
        sum_amount = amount_48htype1_Li + amount_48htype2_Li + amount_24g_Li + amount_weirdo

        # sanity check for the amount
        # if amount_weirdo == nr_of_weirdos_Li & sum_amount == sum_sanitycheck_48htypesmerged_Li_new:
        # if int(amount_weirdo) == int(nr_of_weirdos_Li) & int(sum_amount) == int(sum_sanitycheck_48htypesmerged_Li_new):
        if int(amount_weirdo) == int(nr_of_weirdos_Li):
            if int(sum_amount) == int(sum_sanitycheck_48htypesmerged_Li_new):
                dataframe.at[idx, col_sum_label_and_weirdo_flag] = "True"

        amount_types_and_weirdo = f"48htype1: {amount_48htype1_Li}, 48htype2: {amount_48htype2_Li}, 24g: {amount_24g_Li}, weirdo: {amount_weirdo}"

        # ratio_48htype1_Li = amount_48htype1_Li / sum_amount
        # ratio_48htype2_Li = amount_48htype2_Li / sum_amount
        # ratio_24g_Li = amount_24g_Li / sum_amount
        # ratio_weirdo_Li = amount_weirdo / sum_amount

        dataframe.at[idx, col_idx_coor_limapped_weirdos_dict] = idx_coor_limapped_weirdos_dict
        dataframe.at[idx, col_amount_types_and_weirdo] = amount_types_and_weirdo
        # dataframe.at[idx, col_ratio_48htype1_Li] = ratio_48htype1_Li
        # dataframe.at[idx, col_ratio_48htype2_Li] = ratio_48htype2_Li
        # dataframe.at[idx, col_ratio_24g_Li] = ratio_24g_Li
        # dataframe.at[idx, col_ratio_weirdo_Li] = ratio_weirdo_Li
        dataframe.at[idx, col_sum_amount] = sum_amount
        dataframe.at[idx, col_idx_coor_limapped_weirdos_dict_init] = idx_coor_limapped_weirdos_dict_init
        dataframe.at[idx, col_ndim_coor_reduced120_corrected_Li_new] = ndim_coor_reduced120_corrected_Li_new
        dataframe.at[idx, col_ndim_coor_weirdos_48htypesmerged_Li] = ndim_coor_weirdos_48htypesmerged_Li
        dataframe.at[idx, col_len_coor_weirdos_48htypesmerged_Li] = len_coor_weirdos_48htypesmerged_Li
        dataframe.at[idx, col_len_coor_reduced120_corrected_Li_new] = len_coor_reduced120_corrected_Li_new
        dataframe.at[idx, col_len_idx0_weirdos_Li] = len_idx0_weirdos_Li
        dataframe.at[idx, col_len_idx_without_weirdos] = len_idx_without_weirdos